# Sistemas híbridos


*Autómatos híbridos* são modelos de sistemas ciber-físicos. Essencialmente um autómato híbrido é uma *máquina de estados finita*, onde cada estado (designado por *modo de funcionamento*) descreve o comportamento contínuo de um sistema dinâmico modelado por *relações diferenciais ordinárias* (nas variáveis contínuas e nas suas derivadas em relação ao tempo) codificadas num predicado designado por *flow*. Estas variáveis contínuas evoluem num modo de funcionamento enquanto o seu *flow* for válido.
Cada transição discreta entre estados (um *switch* entre modos de funcionamento) é representada por um arco anotado com um evento e um predicado (*jump*). O *switch* realiza-se quando o seu *jump* é válido. Ao ocorrer uma transição as variáveis internas dos modos conservam o seu valor, a não ser que lhes seja explicitamente atribuído um novo valor.

*Sistemas híbridos* são modelos constituídos por vários autómatos híbridos que, em princípio, evoluem de forma “quase-independente”; a única dependência manifesta-se no sincronismo de duas ou mais transições relativas ao mesmo evento em autómatos distintos.

Um sistema híbrido pode ser descrito por um FOTS (sobre o qual podemos verificar propriedades lógicas com as metodologias que já estudamos). Nesse processo o FOTS vai *discretizar* as relações diferenciais ordinárias e vai mapea-las num espaço de estados também discreto.


### Exemplo do Termostato

Considere o seguinte autómato híbrido que modela o comportamento de um termostato que controla uma caldeira de aquecimento.

<img src="termostato.png" width="500"/>

Inicialmente a temperatura (aqui representada pela variável $x$) é de 20 graus. Quando a temperatura é menor que 19 graus o termostato pode ligar-se. Como o modo em que o termostato está desligado tem o invariante $x>=18$, esta temperatura estabelece um limite a partir do qual o termostato tem necessariamente que se ligar. O termostato só pode voltar a desligar quando a temperatura excede os 21 graus, fazendo-o necessariamente se esta atingir os 22 graus devido ao invariante no modo em que está ligado.


## Verificação de autómatos híbridos com FOTS

Podemos verificar propriedades de segurança de autómatos híbridos codificando a semântica dos mesmos num FOTS e usando depois os procedimentos de verificação implementados nas aulas anteriores.

A ideia básica é incluir nos estados do FOTS para além das variáveis contínuas do autómato híbrido duas variáveis especiais:
- uma variável contínua que denota o tempo ($t$ no caso do termostato)
- uma variável discreta que denota o *modo de funcionamento* ($m$ no caso do termostato)

O estado inicial do FOTS é derivado facilmente a partir da definição do autómato híbrido. Por exemplo, no caso do termostato temos

$$
t = 0 \wedge m = \mathsf{INIT} \wedge x = 20
$$

As transições do FOTS incluem os dois tipos de transição que podem ocorrer num autómato híbrido:
- Transições *timed* descrevem os *flows* associados a cada modo (a evolução das variáveis contínuas)
- Transições *untimed* descrevem os *switches* entre modos

As transições *untimed* podem ser obtidas através de uma codificação muito directa das guardas e efeitos especificadas nos *switches*, com a restrição que o tempo não evolui nestas transições, nem as variáveis contínuas se modificam a não ser que lhes seja explicitamente atribuído um novo valor no efeito do *switch*. Por exemplo, no caso do termostato temos 3 transições deste tipo:

$$
\begin{array}{c}
m = \mathsf{INIT} \wedge m' = \mathsf{OFF} \wedge x' = x \wedge t' = t \\
\vee\\
m = \mathsf{OFF} \wedge m' = \mathsf{ON} \wedge x < 19 \wedge x' = x \wedge t' = t \\
\vee\\
m = \mathsf{ON} \wedge m' = \mathsf{OFF} \wedge x \ge 21 \wedge x' = x \wedge t' = t 
\end{array}
$$

Nas transições *timed* o modo permanece constante, mas o resto das variáveis evoluem de acordo com as restrições indicadas. Os *flows* são especificados indicando qual a derivada em relação ao tempo de cada variável contínua. Para codificar os *flows* no FOTS é necessário fazer a sua *discretização*, ou seja, indicar qual a variação ocorrida num intervalo de tempo $t'-t$. Se a derivada for uma constante a discretização é trivial. Por exemplo, se $\dot{x} = k$ temos que $x' - x = k(t'-t)$. 
Se a relação de *flow* é amortecida, como por exemplo no modo $\mathsf{OFF}$ do termostato, uma pseudo-solução seria aproximar a derivada $\dot{x}$ por $\frac{x'-x}{t'-t}$ obtendo-se
$5(x' - x) + x*(t'-t) = 0$, mas nesta equação ocorre um produto não-escalar $\,x*(t'-t)\,$ e, por isso,  ela não vai ser decidível.
Uma sugestão para discretizar consiste em usar um valor constante para substituir a variável $x$, inferido a partir do invariante de modo. Neste caso a equação de flow transforma-se numa inequação.


Por exemplo, no caso do *flow* associado ao modo $\mathsf{OFF}$, $5\dot{x}+x=0$,  como sabemos que $x \ge 18$ podemos substituir $x$ por 18 na equação, dando origem à relação $x' - x \le -\frac{18}{5}(t'-t)$. 
No caso do *flow* associado ao modo $\mathsf{ON}$, $5\dot{x}+x=25$,  como $x \le 22$ podemos aproximar $x$ por 22, dando origem à relação $x' - x \ge \frac{3}{5}(t'-t)$. 

Finalmente, é necessário também impor os invariantes dos modos no FOTS. Isso pode ser feito acrescentando a cada transição uma restrição que obriga o invariante a ser cumprido. Temos também que assugurar que o tempo avança.


Com esta técnica, no caso do termostato teríamos as seguintes duas transições *timed*:

$$
\begin{array}{c}
m = \mathsf{OFF} \wedge m' = m \wedge x' - x \le -\frac{18}{5}(t'-t) \wedge x \ge 18 \wedge x' \ge 18 \wedge t'>t \\
\vee\\
m = \mathsf{ON} \wedge m' = m \wedge x' - x \ge \frac{3}{5}(t'-t) \wedge x \le 22 \wedge x' \le 22 \wedge t' > t
\end{array}
$$

Para reduzir os erros na verificação pode-se reduzir a granularidade da discretização subdividindo cada modo em vários sub-modos que cubram toda a gama dos valores permitidos. Por exemplo neste caso poderíamos dividir cada um dos modos em 4 sub-modos, cada um com uma variação de temperatura máxima de 1 grau, cobrindo assim toda a gama de temperaturas possíveis (entre 18 e 22 graus).

Na codificação em Z3 é conveniente usar um tipo enumerado para implementar os modos. No caso do termostato podemos declarar esse tipo da seguinte forma.

In [22]:
from z3 import *

Mode, (INIT,ON,OFF) = EnumSort('Mode', ('INIT','ON','OFF'))

Podemos agora declarar as variáveis do FOTS correspondente ao termostato da seguinte forma.

In [23]:
def declare(i):
    s = {}
    s['t'] = Real('t'+str(i))
    s['m'] = Const('m'+str(i),Mode)
    s['x'] = Real('x'+str(i))
    return s

### Exercício 1

Codifique os predicados Z3 `init` e `trans` que caracterizam, respectivamente, os estados iniciais e as transições do FOTS correspondente ao termostato.

In [24]:
def init(s):
    # completar
    return And(s['m'] == INIT, s['t'] == 0, s['x'] == 20)

def trans(s,p):
    # untimed
    initOff = And(s['m'] == INIT, p['m'] == OFF, p['x'] == s['x'], p['t'] == s['t'])
    offOn = And(s['m'] == OFF, p['m'] == ON, s['x'] < 19, p['x'] == s['x'], p['t'] == s['t'])
    onOff = And(s['m'] == ON, p['m'] == OFF, s['x'] >= 21, p['x'] == s['x'], p['t'] == s['t'])
    
    # timed
    offOff = And(s['m'] == OFF, p['m'] == s['m'], p['x']-s['x'] <=(-18/5)*(p['t']-s['t']),s['x'] >= 18, p['x'] >= 18, p['t'] > s['t'])
    onOn = And(s['m'] == ON, p['m'] == s['m'], p['x']-s['x'] >=(3/5)*(p['t']-s['t']),s['x'] <= 22, p['x'] <=22, p['t'] > s['t'])
    
    return Or(initOff,offOn,onOff,offOff,onOn)

### Exercício 2

Adapte a função `gera_traco` implementada nas aulas anterior por forma a imprimir todas as variáveis continuas como números de virgula flutuante. Para saber qual o tipo (*sort*) de uma variável use o método `sort()`. No caso das variáveis contínuas o tipo será `RealSort()`. Para converter o valor `v` de uma variável contínua do Z3 para um float do Python use a expressão `float(v.numerator_as_long())/float(v.denominator_as_long())`.

In [25]:
def gera_traco(declare,init,trans,k):
    s = Solver()
    
    # it's like an authomata
    # declare all k to states
    trace = [declare(i) for i in range(k)]
    
    # initializate state 0
    s.add(init(trace[0]))
    
    # create a link between two spaces
    for i in range(k-1):
        s.add(trans(trace[i], trace[i+1]))
    
    if s.check() == sat:
        m = s.model()
        for i in range(k):
            print(i)
            for v in trace[i]:
                r = m[trace[i][v]]
                if r.sort() == RealSort():
                    print(v, '=', float(r.numerator_as_long())/float(r.denominator_as_long()))
                else:  
                    print(v,'=', r)
                
gera_traco(declare,init,trans,5)

0
t = 0.0
m = INIT
x = 20.0
1
t = 0.0
m = OFF
x = 20.0
2
t = 0.18518518518518517
m = OFF
x = 19.333333333333332
3
t = 0.37037037037037035
m = OFF
x = 18.666666666666668
4
t = 0.5555555555555556
m = OFF
x = 18.0


### Exercício 3

Adapte a função `bmc_always` implementada nas aulas anterior para, tal como no exercício anterior, quando ocorrer um contra-exemplo imprima também todas as variáveis continuas como números de virgula flutuante. 

In [26]:
def bmc_always(declare,init,trans,inv,K):
    for k in range(1,K+1):
        s = Solver()
        # it's like an authomata
        # declare all k to states
        trace = [declare(i) for i in range(k)]

        # initializate state 0
        s.add(init(trace[0]))

        # create a link between two spaces
        for i in range(k-1):
            s.add(trans(trace[i], trace[i+1]))
            
        s.add(Not(inv(trace[k-1])))
        
        if s.check() == sat:
            m = s.model()
            for i in range(k):
                print(i)
                for v in trace[i]:
                    print(v,'=', m[trace[i][v]])
            return
        
    print ("Property is valid up to traces of length "+str(K))
        

def positive(s):
    return s['x'] >= 0

bmc_always(declare,init,trans,positive,10)

Property is valid up to traces of length 10


### Exercício 4

A propriedade "o termostato irá inevitavelmente estar ligado" é uma propriedade de *liveness* exprimível como $F\ (m = \mathsf{ON})$. No entanto a propriedade "passado 1 segundo o termostato irá inevitavelmente estar ligado" já é uma propriedade de *safety*. Explique porquê e verifique também se esta propriedade se verifica no termostato.

In [27]:
def prop(s):
    return Implies(s['t'] == 1, s['m'] == ON)



bmc_always(declare,init,trans,prop,10)

0
t = 0
m = INIT
x = 20
1
t = 0
m = OFF
x = 20
2
t = 5/13
m = OFF
x = 242/13
3
t = 5/13
m = ON
x = 242/13
4
t = 1
m = ON
x = 21
5
t = 1
m = OFF
x = 21


### Exercício 5

Como pode verificar o contra-exemplo obtido não é realista e deve-se à aproximação feita na discretização, que permite mais comportamentos que os realmente possíveis no sistema real. Isto faz com que seja possível termos falsos contra-exemplos para algumas propriedades. Para reduzir estes falsos contra-exemplos podemos tentar melhorar essa discretização, por exemplo, tentando inferir invariantes mais fortes para as variáveis contínuas em cada um dos modos. No caso do termostato sabemos que em qualquer dos modos de facto temos que $18 \le x \le 22$. Usando esta informação refine a definição do predicado `trans` e volte a verificar a propriedade. Se o contra-exemplo continuar a ser claramente inválido podemos refinar mais cada um dos modos, desdobrando-os para gamas da variável $x$ mais apertadas, por exemplo, dividindo cada um deles em 2 sub-modos onde $18 \le x \le 20$ e $20 \le x \le 22$.

In [32]:
def trans(s,p):
    # untimed
    initOff = And(s['m'] == INIT, p['m'] == OFF, p['x'] == s['x'], p['t'] == s['t'])
    offOn = And(s['m'] == OFF, p['m'] == ON, s['x'] < 19, p['x'] == s['x'], p['t'] == s['t'])
    onOff = And(s['m'] == ON, p['m'] == OFF, s['x'] >= 21, p['x'] == s['x'], p['t'] == s['t'])
    
    # timed
    offOff = And(s['m'] == OFF, p['m'] == s['m'], p['x']-s['x'] <=(-18/5)*(p['t']-s['t']),p['x']-s['x'] >=(-22/5)*(p['t']-s['t']),s['x'] >= 18,p['x']>=18, p['x'] <=22, s['x'] <= 22, p['t'] > s['t'])
    
    onOn = And(s['m'] == ON, p['m'] == s['m'], p['x']-s['x'] >=(3/5)*(p['t']-s['t']), p['x']-s['x'] <=(7/5)*(p['t']-s['t']),s['x'] >= 18,p['x']>=18,s['x'] <= 22, p['x'] <=22, p['t'] > s['t'])
    
    return Or(initOff,offOn,onOff,offOff,onOn)

def prop(s):
    return Implies(s['t'] == 1, s['m'] == ON)



bmc_always(declare,init,trans,prop,10)

Property is valid up to traces of length 10


### Exercício 6

Considere o seguinte autómato híbrido que modela um contador que no modo $\mathsf{ON}$ evolui síncronamente com o tempo. Neste caso vamos assumir que a variável de tempo é designada $\tau$ e variável de modo por $n$.

<img src="contador.png" width="500"/>

Modele este autómato com um FOTS em Z3 e verifique que o valor do contador é sempre não superior ao tempo decorrido.

In [33]:
def declare(i):
    s = {}
    s['u'] = Real('u'+str(i))
    s['n'] = Const('n'+str(i),Mode)
    s['c'] = Real('c'+str(i))
    return s

def init(s):
    return And(s['u'] == 0, s['c'] == 0, s['n'] == INIT)
    
def trans(s,p):
    # untimed
    initOff = And(s['n'] == INIT, p['n'] == OFF, p['c'] == s['c'], p['u'] == s['u'])
    offOn = And(s['n'] == OFF, p['n'] == ON, p['c'] == s['c'], p['u'] == s['u'])
    onOff = And(s['n'] == ON, p['n'] == OFF, p['c'] == s['c'], p['u'] == s['u'])
    
    #timed
    offOff = And(s['n'] == OFF, p['n'] == OFF, p['c']-s['c'] == 0*(p['u'] - s['u']), p['u'] > s['u'])
    onOn = And(s['n'] == ON, p['n'] == ON, p['c']-s['c'] == 0*(p['u'] - s['u']), p['u'] > s['u']) # falta alterar coisas
    
    return Or(initOff,offOn,onOff,offOff,onOn)

def slower(s):
    return s['c'] <= s['u']

    
bmc_always(declare,init,trans,slower,10)

Property is valid up to traces of length 10


## Verificação de sistemas híbridos com FOTS

Se quisermos verificar propriedades sobre um sistema híbrido, por exemplo, sobre o sistema que incluiu os dois autómatos acima, temos que modelar a evolução de todos os seus autómatos num único FOTS. Os estados deste FOTS correspondem ao produto cartesiano dos estados dos vários autómatos, sendo o estado inicial caracterizado pela conjunção dos predicados iniciais de todos eles. Por exemplo, num sistema híbrido com um termostato e um contador,  o estado inicial seria caracterizado pelo seguinte predicado.

$$
m = \mathsf{INIT} \wedge x = 20 \wedge t = 0 \wedge n = \mathsf{INIT} \wedge c = 0 \wedge \tau = 0
$$

Para tal temos que considerar como é que as transições dos diferentes autómatos estão sincronizadas:

- as transições *timed* de diferentes autómatos ocorrem simultaneamente sse o intervalo de tempo decorrido for igual em todos os *flows*
- as transições *untimed* de diferentes autómatos ocorrem simultaneamente sse o evento que ocorre for igual em todos os *switches*

No sistema híbrido exemplo, a transição *untimed* relativa ao evento $\mathsf{init}$ seria caracterizada pela seguinte restrição.

$$
m = \mathsf{INIT} \wedge m' = \mathsf{OFF} \wedge x' = x \wedge t' = t \wedge n = \mathsf{INIT} \wedge n' = \mathsf{OFF} \wedge c' = c \wedge \tau' = \tau
$$

No caso das transições *timed* temos que considerar todas as combinações de transições *timed* nos diferentes modos. Por exemplo, quando os dois autómatos estão no estado $\mathsf{OFF}$ o *flow* pode ser caracterizado pela seguinte restrição (sem considerar o refinamento introduzido no exercício 5).

$$
\begin{array}{c}
m = \mathsf{OFF} \wedge m' = m \wedge x' - x \le -\frac{18}{5}(t'-t) \wedge x \ge 18 \wedge x' \ge 18 \wedge t'>t\\
\wedge\\
n = \mathsf{OFF} \wedge n' = n \wedge c' - c = 0 \wedge \tau' > \tau\\
\wedge\\
t' - t = \tau' - \tau
\end{array}
$$

### Exercício 7

Modele este sistema híbrido em Z3 e gere um traço com 5 estados.

In [ ]:
# completar 

### Exercício 8

Verifique que o contador só é positivo passados pelo menos 0.2 segundos.

In [ ]:
# completar 